In [3]:
from subprocess import check_output
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

warnings.filterwarnings('ignore')
num_classes = 10
epochs = 20
class_names = ['T_shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_df = pd.read_csv('./input/fashion-mnist_train.csv', sep=',')
test_df = pd.read_csv('./input/fashion-mnist_test.csv', sep = ',')

# print(check_output(["ls", "./input"]).decode("utf8"))
# print(train_df.head())
print(train_df.shape)


(60000, 785)


## Residual Learning
RL (Residual Learning) to technika stosowana w sieciach neuronowych, w szczególności w warstwach konwolucyjnych, która pozwala na zwiększenie wydajności uczenia poprzez zmniejszenie problemu gradientu zanikającego (ang. vanishing gradient problem).

Warstwa RL (Residual Layer) składa się z bloku resztkowego (ang. residual block), który dodaje oryginalne dane wejściowe do wyników operacji konwolucji i normalizacji, a następnie poddaje je funkcji aktywacji. Dzięki temu warstwa RL uczy się reszty (ang. residual), czyli różnicy pomiędzy oryginalnymi danymi wejściowymi a wynikami operacji konwolucji. W przypadku, gdy blok resztkowy nie będzie w stanie nauczyć się reszty, model wciąż będzie miał dostęp do oryginalnych danych wejściowych, co poprawia wydajność uczenia i zwiększa dokładność modelu.

Warstwy RL są szczególnie przydatne przy budowie bardzo głębokich sieci neuronowych, w których problem gradientu zanikającego może znacznie wpłynąć na wydajność uczenia. Warstwy RL są stosowane w sieciach konwolucyjnych, ale również w innych rodzajach sieci neuronowych, takich jak sieci ResNet i DenseNet.

## Same Padding
"Same padding" to jedna z technik używanych w sieciach konwolucyjnych (CNN), która polega na uzupełnianiu danych wejściowych tak, aby wyjście miało tę samą wymiarowość co wejście.

W przypadku konwolucji w sieciach neuronowych, wymiarowość wyjściowego obrazu zależy od rozmiaru filtra konwolucyjnego i sposobu, w jaki filtry są przesuwane po obrazie wejściowym. Bez uzupełniania danych, podczas stosowania filtrów konwolucyjnych, wyjście z każdej warstwy konwolucyjnej zmniejszałoby się, co prowadziłoby do utraty informacji i pogorszenia dokładności modelu.

W technice same padding, dane wejściowe są uzupełniane tak, aby ich wymiarowość po przetworzeniu przez filtr konwolucyjny pozostawała niezmieniona. W przypadku filtrów o nieparzystych rozmiarach, takich jak 3x3, 5x5 itd., padding jest dodawany symetrycznie z obu stron wejściowego obrazu. Natomiast w przypadku filtrów o parzystych rozmiarach, takich jak 2x2, 4x4 itd., padding jest dodawany z jednej strony wejściowego obrazu.

Technika same padding ma na celu zapobieganie utracie informacji i zapewnienie, że wyjście z każdej warstwy konwolucyjnej będzie miało tę samą wymiarowość co wejście, co ułatwia tworzenie głębszych sieci neuronowych i poprawia jakość klasyfikacji lub segmentacji obrazu.

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, Flatten, Dense
from keras.models import Model
from keras.utils import to_categorical
import time

start = time.time()

x_train = train_df.iloc[:, 1:].values.reshape(-1, 28, 28, 1)
y_train = to_categorical(train_df.iloc[:, 0].values, 10)
x_test = test_df.iloc[:, 1:].values.reshape(-1, 28, 28, 1)
y_test = to_categorical(test_df.iloc[:, 0].values, 10)

x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.2, random_state = 12345)


# Definicja bloku resztkowego
def residual_block(inputs, filters):
    x = Conv2D(filters, kernel_size=(3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Add()([x, inputs])
    x = Activation('relu')(x)
    return x

inputs = Input(shape=(28, 28, 1))
x = Conv2D(64, kernel_size=(3, 3), padding='same')(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = residual_block(x, 64)
x = residual_block(x, 64)

x = Conv2D(128, kernel_size=(3, 3), padding='same', strides=(2, 2))(x)
x = residual_block(x, 128)
x = residual_block(x, 128)

x = Conv2D(256, kernel_size=(3, 3), padding='same', strides=(2, 2))(x)
x = residual_block(x, 256)
x = residual_block(x, 256)

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
outputs = Dense(10, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=64, epochs=3, validation_data=(x_validate, y_validate))

loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('===========================')
print('Test loss:', loss)
print('Test accuracy:', accuracy)
print('===========================')
diff = time.time() - start
print('It took me {:.2f} mins'.format(diff/60))

Epoch 1/3


2023-03-06 15:36:49.814783: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


750/750 [==============================] - ETA: 0s - loss: 0.8848 - accuracy: 0.7626

2023-03-06 15:38:50.500614: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


750/750 [==============================] - 131s 172ms/step - loss: 0.8848 - accuracy: 0.7626 - val_loss: 0.3727 - val_accuracy: 0.8650
Epoch 2/3
750/750 [==============================] - 129s 172ms/step - loss: 0.3329 - accuracy: 0.8786 - val_loss: 0.3018 - val_accuracy: 0.8905
Epoch 3/3
750/750 [==============================] - 127s 170ms/step - loss: 0.2798 - accuracy: 0.8968 - val_loss: 0.2970 - val_accuracy: 0.8899
Test loss: 0.28871530294418335
Test accuracy: 0.8927000164985657
It took me 6.60 mins


In [8]:
from sklearn.metrics import classification_report

pred = model.predict(x_test)
predicted_classes = np.argmax(pred, axis=-1)
y_test = test_df.iloc[:, 0]

print(classification_report(y_test, predicted_classes, target_names=class_names))

313/313 [==============================] - 8s 24ms/step
              precision    recall  f1-score   support

 T_shirt/top       0.90      0.76      0.82      1000
     Trouser       0.99      0.98      0.99      1000
    Pullover       0.87      0.78      0.82      1000
       Dress       0.92      0.93      0.92      1000
        Coat       0.79      0.90      0.84      1000
      Sandal       0.94      0.98      0.96      1000
       Shirt       0.74      0.74      0.74      1000
     Sneaker       0.97      0.90      0.93      1000
         Bag       0.90      1.00      0.95      1000
  Ankle boot       0.94      0.97      0.95      1000

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

